# Implementation of ChatBot using Python and NLTK

In [ ]:
pip install nltk scikit-learn streamlit

In [ ]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

In [ ]:
import json

# Define the intents data as a Python dictionary
intents = {
    "intents": [
        {
            "tag": "book_appointment",
            "patterns": [
                "I want to book an appointment",
                "Can I schedule a visit with a doctor?",
                "I need to see a doctor on Monday",
                "Book an appointment for me at 10 AM tomorrow"
            ],
            "responses": [
                "Sure! Please provide the doctor's name, preferred date, and time.",
                "Which department or specialist do you want to book an appointment with?"
            ]
        },
        {
            "tag": "check_availability",
            "patterns": [
                "What are the available slots for Dr. John?",
                "Is there any availability on Friday?",
                "When can I get an appointment with a cardiologist?"
            ],
            "responses": [
                "Dr. John is available on [date] at [time]. Would you like to confirm the booking?",
                "There are available slots on [date]. Would you like me to book it for you?"
            ]
        },
        {
            "tag": "virtual_consultation",
            "patterns": [
                "Can I book a virtual consultation?",
                "I want an online doctor appointment",
                "Can I consult a doctor via video call?",
                "Schedule a telemedicine appointment"
            ],
            "responses": [
                "Sure! Would you prefer a video call or chat-based consultation?",
                "We offer virtual consultations via video call and chat. Which one do you prefer?"
            ]
        },
        {
            "tag": "video_call_appointment",
            "patterns": [
                "I want a video consultation",
                "Can I meet the doctor on a video call?",
                "Schedule a video appointment with a doctor"
            ],
            "responses": [
                "I can schedule a video call with Dr. [Name]. Please provide a preferred date and time.",
                "Dr. [Name] is available for a video consultation on [date] at [time]. Would you like to confirm?"
            ]
        },
        {
            "tag": "chat_based_consultation",
            "patterns": [
                "I prefer a chat consultation",
                "Can I talk to a doctor via chat?",
                "I want to consult through messages"
            ],
            "responses": [
                "Sure! You can start chatting with Dr. [Name] now.",
                "Dr. [Name] is available for chat consultation. Connecting you now."
            ]
        },
        {
            "tag": "prescription_reports",
            "patterns": [
                "Where can I see my prescriptions?",
                "Can I download my medical reports?",
                "Send me my latest prescription"
            ],
            "responses": [
                "Your prescription from Dr. [Name] is available in your account. Would you like me to send it to your email?",
                "I have fetched your latest prescription. You can download it here: [link]."
            ]
        },
        {
            "tag": "online_payment",
            "patterns": [
                "How can I pay for my virtual consultation?",
                "Do you accept online payments?",
                "Can I pay using UPI or credit card?"
            ],
            "responses": [
                "You can pay via UPI, credit/debit card, or net banking. Would you like the payment link?",
                "Your virtual consultation fee is $50. Click here to complete the payment: [link]."
            ]
        },
        {
            "tag": "modify_appointment",
            "patterns": [
                "I need to reschedule my appointment",
                "Can I change my booking to another time?",
                "Cancel my appointment for tomorrow"
            ],
            "responses": [
                "Your appointment is currently scheduled for [date, time]. Would you like to reschedule or cancel?",
                "What new date and time would you prefer?"
            ]
        },
        {
            "tag": "doctor_info",
            "patterns": [
                "Tell me about Dr. Smith",
                "Who are the available dermatologists?",
                "I need a doctor who specializes in neurology"
            ],
            "responses": [
                "Dr. Smith is a cardiologist with 10 years of experience. Would you like to book an appointment?",
                "We have Dr. [Name] available for neurology. Would you like to schedule a consultation?"
            ]
        },
        {
            "tag": "walkin_appointment",
            "patterns": [
                "Can I visit the clinic without an appointment?",
                "Do you accept walk-in patients?",
                "I need to see a doctor without booking"
            ],
            "responses": [
                "Yes, walk-ins are accepted. However, scheduled appointments have priority. Would you like me to check the estimated wait time?",
                "You can visit the clinic as a walk-in. Let me check the current queue."
            ]
        },
        {
            "tag": "billing_inquiry",
            "patterns": [
                "How much does a consultation cost?",
                "What are the payment options?",
                "Can I pay online for my appointment?"
            ],
            "responses": [
                "A consultation costs $50. You can pay online or at the clinic.",
                "We accept credit/debit cards, insurance, and online payments."
            ]
        },
        {
            "tag": "patient_records",
            "patterns": [
                "Can you access my medical records?",
                "What details do you have about my last visit?",
                "Do you have my patient history?"
            ],
            "responses": [
                "Your last visit was on [date] with Dr. [Name]. Would you like me to retrieve your medical records?",
                "Yes, I can fetch your patient history. Do you need a summary?"
            ]
        },
        {
            "tag": "general_help",
            "patterns": [
                "How does this system work?",
                "What can you do?",
                "Help me use this chatbot"
            ],
            "responses": [
                "I can help you book, modify, or cancel appointments, check doctor availability, and provide clinic details. How can I assist you today?"
            ]
        }
    ]
}

# Save intents to a file
with open("intents.json", "w") as file:
    json.dump(intents, file)

print("intents.json file has been saved successfully.")


In [3]:
import json
import random

# Define the chatbot code as a string
chatbot_code = """
import json
import random

# Load the intents file
with open("intents.json", "r") as file:
    intents = json.load(file)

# Function to get chatbot response
def get_response(user_input):
    user_input = user_input.lower()
    for intent in intents["intents"]:
        for pattern in intent["patterns"]:
            if pattern.lower() in user_input:
                return random.choice(intent["responses"])
    return "I'm sorry, I didn't understand that. Can you please rephrase?"

# Function to display available services when chatbot starts
def show_available_services():
    print("\\nWelcome to the AI Healthcare Chatbot! 🏥")
    print("Here are the services I can help you with:\\n")
    services = [
        "📅 Book an appointment",
        "📆 Check doctor availability",
        "💻 Virtual consultation (Video Call / Chat)",
        "📝 View prescriptions & medical records",
        "💳 Online payment for consultation",
        "🚑 Emergency appointment booking",
        "🚶 Walk-in appointment status",
        "🔔 Appointment reminders",
        "📊 Get past appointment reports",
        "💰 Billing & payment information",
        "🩺 Get doctor information",
        "❓ General help & FAQs"
    ]
    for service in services:
        print(f"- {service}")
    print("\\nType your request below 👇")

# Start chatbot with services list
show_available_services()

# Run chatbot loop
while True:
    user_input = input("\\nYou: ")
    if user_input.lower() == "exit":
        print("Goodbye! Stay healthy! 😊")
        break
    response = get_response(user_input)
    print("Bot:", response)
"""

# Save the Python code to a .py file with UTF-8 encoding
with open("chatbot.py", "w", encoding="utf-8") as file:
    file.write(chatbot_code)

print("chatbot.py file has been saved successfully.")


chatbot.py file has been saved successfully.


In [4]:
# Define the Streamlit chatbot code as a string
streamlit_code = """
import streamlit as st
import json
import random

# Load the intents file
with open("intents.json", "r") as file:
    intents = json.load(file)

# Function to get chatbot response
def get_response(user_input):
    user_input = user_input.lower()
    for intent in intents["intents"]:
        for pattern in intent["patterns"]:
            if pattern.lower() in user_input:
                return random.choice(intent["responses"])
    return "I'm sorry, I didn't understand that. Can you please rephrase?"

# Function to display available services when chatbot starts
def show_available_services():
    services = [
        "📅 Book an appointment",
        "📆 Check doctor availability",
        "💻 Virtual consultation (Video Call / Chat)",
        "📝 View prescriptions & medical records",
        "💳 Online payment for consultation",
        "🚑 Emergency appointment booking",
        "🚶 Walk-in appointment status",
        "🔔 Appointment reminders",
        "📊 Get past appointment reports",
        "💰 Billing & payment information",
        "🩺 Get doctor information",
        "❓ General help & FAQs"
    ]
    return services

# Streamlit app layout
st.title("AI Healthcare Chatbot 🤖")
st.write("Welcome to the AI Healthcare Chatbot! 🏥")

# Show available services
st.write("Here are the services I can help you with:")
services = show_available_services()
for service in services:
    st.write(f"- {service}")

# Text input for user interaction
user_input = st.text_input("You:", "")

if user_input:
    # Get response from chatbot
    response = get_response(user_input)
    st.write(f"**Bot:** {response}")
"""

# Save the Streamlit code to a .py file with UTF-8 encoding
with open("app.py", "w", encoding="utf-8") as file:
    file.write(streamlit_code)

print("app.py file has been saved successfully.")


app.py file has been saved successfully.
